<img style="float: left;" src="figures/Perm.png" width="20%">   

# <font color='Red'> $\;$ Reservoir model for geothermal doublet</font>

First we are going to install open-darts (only need to run it once):

In [ ]:
pip install open-darts

## <font color='Blue'>Introduction</font>
###  In this exercise, we run simulation in a 3D channelized reservoir.

## <font color='blue'>The objectives:</font>
1. Introduce custom <font color='red'>'Model'</font> class based on 
 * Class [DartsModel](https://gitlab.com/open-darts/open-darts/-/blob/development/darts-package/darts/models/darts_model.py) with base model capabilities
 * Class [StructReservoir](https://gitlab.com/open-darts/open-darts/-/blob/development/darts-package/darts/models/reservoirs/struct_reservoir.py) with structured reservoir
 * Class [GeothermalPhysics](https://gitlab.com/open-darts/open-darts/-/blob/development/darts-package/darts/models/physics/geothermal.py) for geothermal engine is used
 * Class [PropertyContainer](https://gitlab.com/open-darts/open-darts/-/blob/development/darts-package/darts/models/physics_sup/property_container.py) for allocating property evaluator needed in the simulation
 * Class [GeothermalProperties](https://gitlab.com/open-darts/open-darts/-/blob/development/darts-package/darts/models/physics/iapws/iapws_property.py) for geothermal fluid based on IAPWS-97 Equatipn of State.
2. Use run_python procedure to control run from the python script
3. Introduce wells and change their location

<img style="float: left;" src="slides/Slide15.PNG" width="60%">

<img style="float: left;" src="slides/Slide16.PNG" width="60%">

<img style="float: left;" src="slides/Slide17.PNG" width="60%">

In [ ]:
from darts.models.reservoirs.struct_reservoir import StructReservoir
from darts.models.physics.geothermal import Geothermal
from darts.models.darts_model import DartsModel
from darts.models.physics.iapws.iapws_property_vec import _Backward1_T_Ph_vec
from darts.tools.keyword_file_tools import load_single_keyword
from darts.engines import redirect_darts_output
import numpy as np
redirect_darts_output('run_geothermal.log')

## <font color='Blue'>Brief Introduction of model inheritance:</font>

* Here create the <font color='red'>'Model' </font>  class, which inherits from <font color='red'>DartsModel</font> (the base class).
* It keeps all the functionalities of <font color='red'>DartsModel</font> and can also be extended to add more functionalities.
* If a function is redefined in subclass, the function in base class with identical name will be overridden.

In [ ]:
class Model(DartsModel):
    def __init__(self, n_points=128):
        # call base class constructor
        super().__init__()
        
        self.timer.node["initialization"].start()
        
        # parameters for the reservoir
        (nx, ny, nz) = (60, 60, 3)
        nb   = nx * ny * nz
        perm = np.ones(nb) * 2000
        #perm = load_single_keyword('perm.in', 'PERMX')
        perm = perm[:nb]
        
        poro = np.ones(nb) * 0.2
        self.dx = 30
        self.dy = 30
        dz   = np.ones(nb) * 30
        #perm[:nx*ny] = 1e-5
        #poro[:nx*ny] = 1e-5
                
        # discretize structured reservoir
        self.reservoir = StructReservoir(self.timer, nx=nx, ny=ny, nz=nz, dx=self.dx, dy=self.dy, dz=dz, permx=perm,
                                         permy=perm, permz=perm*0.1, poro=poro, depth=2000)
        
        # add open boundaries
        self.reservoir.set_boundary_volume(xz_minus=1e8, xz_plus=1e8, yz_minus=1e8, yz_plus=1e8)
                                           
        # add well's locations
        self.jw = [30, 30]
        self.iw = [14, 46]
        
        # add well
        self.reservoir.add_well("INJ")
        n_perf = nz        
        # add perforations to te payzone
        for n in range(1, n_perf):
            self.reservoir.add_perforation(well=self.reservoir.wells[-1], i=self.iw[0], j=self.jw[0], k=n+1, 
                                           well_radius=0.16)

        # add well
        self.reservoir.add_well("PRD")
        # add perforations to te payzone        
        for n in range(1, n_perf):
            self.reservoir.add_perforation(self.reservoir.wells[-1], self.iw[1], self.jw[1], n+1, 0.16)

        # rock heat capacity and rock thermal conduction
        hcap = np.array(self.reservoir.mesh.heat_capacity, copy=False)
        rcond = np.array(self.reservoir.mesh.rock_cond, copy=False)
        hcap.fill(2200)
        rcond.fill(500)

        # create pre-defined physics for geothermal
        self.physics = Geothermal(self.timer, n_points, 1, 351, 1000, 10000, cache=False)

        # timestep parameters
        self.params.first_ts = 1e-3
        self.params.mult_ts  = 2
        self.params.max_ts   = 365

        # nonlinear and linear solver tolerance
        self.params.tolerance_newton = 1e-2

        self.timer.node["initialization"].stop()

    def set_initial_conditions(self):
        # initialization with constant pressure and temperature
        self.physics.set_uniform_initial_conditions(self.reservoir.mesh, uniform_pressure=200,
                                                    uniform_temperature=350)

    def set_boundary_conditions(self):
        # activate wells with rate control for inejctor and producer
        for i, w in enumerate(self.reservoir.wells):
            if 'INJ' in w.name:
                w.control = self.physics.new_rate_water_inj(4000, 300)
            else:
                w.control = self.physics.new_rate_water_prod(4000)
                
    def export_pro_vtk(self, file_name='Results'):
        # connect to simulation array
        X = np.array(self.physics.engine.X, copy=False)
        nb = self.reservoir.mesh.n_res_blocks
        # compute temperature using pressure and enthalpy (in different units)
        temp = _Backward1_T_Ph_vec(X[0:2 * nb:2] / 10, X[1:2 * nb:2] / 18.015)
        # define additional arrays to the output
        local_cell_data = {'Temperature': temp,
                           'Perm': self.reservoir.global_data['permx']}
        # use export_vtk defined in the base class (DartsModel)
        self.export_vtk(file_name, local_cell_data=local_cell_data)

## <font color='Blue'>Now we can run the model:</font>

In [ ]:
m = Model()
m.init()

# output initial conditions
m.export_pro_vtk()
m.run_python(1e-3)

for t in range(3):
    # run and output every 10 years (30 in total)
    m.run_python(10*365, restart_dt=365)
    m.export_pro_vtk()

# print timers and statistics for the run
m.print_timers()
m.print_stat()

In [ ]:
import pandas as pd
# output well information to Excel file
td = pd.DataFrame.from_dict(m.physics.engine.time_data)
writer = pd.ExcelWriter('well_data.xlsx')
td.to_excel(writer, 'Sheet1')
writer.save()

In [ ]:
import matplotlib.pyplot as plt
# plot temperature at production well and technological limit
string = 'PRD : temperature'
ax1 = td.plot(x='time', y=[col for col in td.columns if string in col])
ax1.plot([0, 1.1e4],[348, 348])
ax1.tick_params(labelsize=14)
ax1.set_xlabel('Days', fontsize=14)
ax1.legend(['temp', 'limit'], fontsize=14)
plt.grid()
plt.show()

In [ ]:
import pyvista as pv

# get vts data
mesh = pv.read('vtk_data\\results_ts3.vts')

# define plotter
plotter = pv.Plotter()

# set temperature as active scalar
mesh.set_active_scalars('Temperature')

# add threshold levels
thresT = mesh.threshold([300, 348], invert=False)

# add outline of mesh
outline = mesh.outline()

# add elements to plotter
plotter.set_background('#52576c')
plotter.add_mesh(outline, color='k')
plotter.add_mesh(thresT, cmap='coolwarm',
                 stitle='Temperature (\N{DEGREE SIGN}C)')

mesh.set_active_scalars('Perm')
# threshold for plotting permeability map
thresperm = mesh.threshold([1, 7000], scalars='Perm',
                           continuous=True)
# plot permebility map with opacity
plotter.add_mesh(thresperm, scalars='Perm', cmap='viridis',
                 opacity=0.25,
                 stitle='Permeability (mD)')

# add wells as lines
ix_coord = (m.iw[0] - 0.5) * m.dx
iy_coord = (m.jw[0] - 0.5) * m.dy
px_coord = (m.iw[1] - 0.5) * m.dx
py_coord = (m.jw[1] - 0.5) * m.dy

injline = np.array([[ix_coord, iy_coord, -1700], [ix_coord, iy_coord, -2100]])
prodline = np.array([[px_coord, py_coord, -1700], [px_coord, py_coord, -2100]])

_ = plotter.add_lines(injline, color='b', name='injector')
_ = plotter.add_lines(prodline, color='r', name='producer')
_ = plotter.add_axes(line_width=5, labels_off=False)

plotter.show()

## <font color='Blue'>Tasks in this workshop:</font>

1. Load 'PERMX' keyword from file 'perm.in'
2. Rerun the simulation, compare lifetime with homegeneous case
3. Change location of the wells to cross the channels (instead of along the channels as now)
4. Rerun and compare the lifetime, explain why it is different